In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [2]:
website = 'https://in.indeed.com/'
path = Service('/chromedriver.exe')  #path to chromedriver
driver = webdriver.Chrome(service=path)
driver.get(website)
driver.maximize_window()

In [3]:
container=driver.find_element(By.XPATH,"//*[@id='jobsearch']")
job_title=container.find_element(By.XPATH,".//*[@id='text-input-what']")
title=input("job title/keyword:")
job_title.send_keys(title)
submit=container.find_element(By.XPATH,"//button[@type='submit']")
submit.click()

job title/keyword:Data Science


In [4]:
search_result=driver.find_element(By.XPATH,"//ul[contains(@class,'jobsearch-ResultsList')]")
boxes=search_result.find_elements(By.XPATH,".//td[contains(@class,'resultContent')]")

In [5]:
designation=[]
company=[]
location=[]
rating=[]
joblink=[]
salary=[]

In [6]:
last_page = 5 #mention number of pages to scrape
current_page = 1

while current_page < last_page:

    left_panel = driver.find_element(By.XPATH,"//div[@class='jobsearch-LeftPane']")
    single_job_snippet =  left_panel.find_elements(By.XPATH,".//td[@class='resultContent']")



    for single_job in single_job_snippet:
        location.append(single_job.find_element(By.XPATH,".//div[@class='companyLocation']").text)
        designation.append(single_job.find_element(By.XPATH,".//h2").text)
        company.append(single_job.find_element(By.XPATH,".//span[@class='companyName']").text)
        try:
            rating.append(single_job.find_element(By.XPATH,".//a[contains(@class,'ratingLink')]/span").text)
        except:
            rating.append("No Rating Found")
        joblink.append((single_job.find_element(By.XPATH,".//h2/a")).get_attribute('href'))

        #click on job profile
        single_job.click()
        time.sleep(2)

        #fetch job details
        right_panel = driver.find_element(By.XPATH,"//div[@class='jobsearch-RightPane']")
        try:
            salary.append(right_panel.find_element(By.XPATH,".//div[@id='salaryInfoAndJobType']/span").text)
        except:
            salary.append('Not Mentioned')

    try:
        current_page = current_page + 1 
        nxt_btn = driver.find_element(By.XPATH,'//a[contains(@data-testid,"pagination-page-next")]')
        nxt_btn.click()   
        
    except:
        pass
    
    
driver.quit()

In [7]:
df=pd.DataFrame({'Designation':designation,'Company':company,'Loation':location,'Rating':rating,'Salary':salary,'Job Application Link':joblink})
df.to_excel('jobs.xlsx')